In [1]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.0 requires transformers[sentencepiece]<4.39.0,>=4.38.0, but you have transformers 4.41.2 which is incompatible.
autogluon-timeseries 1.1.0 requires transformers[sentencepiece]<4.39.0,>=4.38.0, but you have transformers 4.41.2 which is incompatible.
optimum 1.18.1 requires transformers[sentencepiece]<4.40.0,>=4.26.0, but you have transformers 4.41.2 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.0 re

In [2]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface . llms import HuggingFacePipeline
from langchain . memory import ConversationBufferMemory
from langchain_community . chat_message_histories import ChatMessageHistory
from langchain_community . document_loaders import PyPDFLoader ,TextLoader
from langchain . chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core . runnables import RunnablePassthrough
from langchain_core . output_parsers import StrOutputParser
from langchain import hub

2024-06-24 21:50:59.588219: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 21:51:00.071542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 21:51:00.635664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
Loader = PyPDFLoader
file_path = "../data/YOLOv10_Tutorials.pdf"
loader = Loader(file_path)
documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter ( chunk_size =1000 , chunk_overlap =100)

In [5]:
docs = text_splitter . split_documents ( documents )

print (" Number of sub - documents : ", len ( docs ) )
print ( docs [0])

 Number of sub - documents :  33
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạ

In [6]:
embedding = HuggingFaceEmbeddings ()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
vector_db = Chroma . from_documents ( documents =docs ,embedding = embedding )
retriever = vector_db . as_retriever ()

In [8]:
result = retriever . invoke ("What is YOLO ?")
print ( result )

[Document(page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn\n1! mkdir datasets\n2! unzip -q "/ content / PlantDocv4 . zip" -d "/ content / datasets / PlantDocv4\n/"\n3!rm / content / PlantDocv4 .zip\nQuan sát thư mục giải nén, có thể thấy bộ dữ liệu này đã được gán nhãn và đưa vào format\ncấu trúc dữ liệu training theo yêu cầu của YOLO. Vì vậy, chúng ta sẽ không cần thực hiện\nbước chuẩn bị dữ liệu ở bài này.\n2.Cài đặt và import các thư viện cần thiết: Tương tự như phần trước, các bạn chạy\ncác đoạn code sau để cài đặt các gói thư viện để sử dụng được YOLOv10:\n1!git clone https :// github .com/THU -MIG/ yolov10 .git\n2%cd yolov10\n3!pip install -q -r requirements .txt\n4!pip install -e .\n3.Khởi tạo mô hình YOLOv10: Chúng ta sẽ khởi tạo mộ hình YOLOv10 với phiên\nbảnnano (n) từ trọng số đã được huấn luyện trên bộ dữ liệu COCO. Để tải trọng số\nyolov10n.pt, các bạn chạy đoạn code sau:\n1! wget https :// github .com/THU -MIG/ yolov10 / releases / download /v1 .1/\nyolov10n .pt', meta

In [9]:
nf4_config = BitsAndBytesConfig (
load_in_4bit =True ,
bnb_4bit_quant_type ="nf4",
bnb_4bit_use_double_quant =True ,
bnb_4bit_compute_dtype = torch . bfloat16
)

In [10]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

In [11]:
model = AutoModelForCausalLM . from_pretrained (
MODEL_NAME ,
quantization_config = nf4_config ,
low_cpu_mem_usage = True
)

tokenizer = AutoTokenizer . from_pretrained ( MODEL_NAME )

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/daoan/Projects/AIO-2

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [12]:
model_pipeline = pipeline(
 "text-generation",
 model =model ,
 tokenizer = tokenizer ,  max_new_tokens =512 ,
 pad_token_id = tokenizer . eos_token_id ,
 device_map ="auto"
)
llm = HuggingFacePipeline (
 pipeline = model_pipeline ,
)

In [13]:
prompt = hub. pull ("rlm/rag-prompt")

def format_docs ( docs ) :
 return "\n\n". join (doc . page_content for doc in docs )

rag_chain = (
 {"context": retriever | format_docs , "question": RunnablePassthrough
() }
  | prompt
  | llm
  | StrOutputParser ()
 )

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output . split ("Answer:") [1]. strip ()
print ( answer )

/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tải trọng số từ GitHub và được khởi tạo bằng cách sử dụng thư viện ultralytics.


In [14]:
!pip install -q langchain-openai==0.1.9
!pip install -q langchain_huggingface==0.0.3
!pip install -q chainlit==1.1.304
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!npm install -g localtunnel
!pip install -q numpy==1.24.4


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.0 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 0.31.0 which is incompatible.
autogluon-multimodal 1.1.0 requires transformers[sentencepiece]<4.39.0,>=4.38.0, but you have transformers 4.41.2 which is incompatible.
autogluon-timeseries 1.1.0 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 0.31.0 which is incompatible.
autogluon-timeseries 1.1.0 requires transformers[sentencepiece]<4.39.0,>=4.38.0, but you have transformers 4.41.2 which is incompatible.
optimum 1.18.1 requires transformers[sentencepiece]<4.40.0,>=4.26.0, but you have transformer

2024-06-24 22:23:55 - Created default config file at /home/daoan/Projects/AIO-2024/Module1/Week4/.chainlit/config.toml
2024-06-24 22:23:55 - Created default translation directory at /home/daoan/Projects/AIO-2024/Module1/Week4/.chainlit/translations
2024-06-24 22:23:55 - Created default translation file at /home/daoan/Projects/AIO-2024/Module1/Week4/.chainlit/translations/en-US.json


2024-06-24 22:23:55 - Use pytorch device_name: cuda
2024-06-24 22:23:55 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/daoan/Projects/AIO-2024/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/daoan/Projects/AIO-2